In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)

## Data Exploration
In this notebook, we try and create the image embedding by first extracting features from the histology image using the pretrained ResNet50, and then using PCA to reduce the image down to the desired dimensionality.


In [ ]:
# cell type and classification
coords = pd.read_csv('bucket/BC23209_C1_Coords.tsv.gz', compression = 'gzip', sep = '\t')
coords.columns = ['records'] +  list(coords.columns[1:])
coords

,records,ycoord,lab,tumor,Unnamed: 4
0,C1_18.921_4.999,18.921,4.999,L1,non
1,C1_19.973_4.997,19.973,4.997,L2,tumor
2,C1_17.887_5.988,17.887,5.988,L3,tumor
3,C1_18.931_5.99,18.931,5.990,L4,non
4,C1_19.986_6.024,19.986,6.024,L5,tumor
...,...,...,...,...,...
289,C1_8.984_25.971,8.984,25.971,L290,tumor
290,C1_7.91_25.988,7.910,25.988,L291,tumor
291,C1_6.908_25.994,6.908,25.994,L292,tumor
292,C1_7.929_26.972,7.929,26.972,L293,tumor


In [ ]:
# gene expression by cell
stdata = pd.read_csv('bucket/BC23209_C1_stdata.tsv.gz', compression = 'gzip', sep = '\t')
stdata.columns = ['records'] +  list(stdata.columns[1:])
stdata

,records,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,__ambiguous[ENSG00000280893+ENSG00000281348+ENSG00000280789],__ambiguous[ENSG00000281039+ENSG00000006625],__ambiguous[ENSG00000281039+ENSG00000196295],__ambiguous[ENSG00000281593+ENSG00000180233],__ambiguous[ENSG00000281887+ENSG00000196329],__ambiguous[ENSG00000281887+ENSG00000213203],__ambiguous[ENSG00000281991+ENSG00000282034],__ambiguous[ENSG00000282246+ENSG00000165630],__ambiguous[ENSG00000282278+ENSG00000145216],__ambiguous[ENSG00000282558+ENSG00000112941]
0,3x34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3x30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3x31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3x32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30x25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,29x7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,29x5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,19x9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,16x24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# spatial coordinates of cell
spots = pd.read_csv('bucket/spots_BC23209_C1.csv.gz', compression = 'gzip', sep = ',')
spots.columns = ['records'] +  list(spots.columns[1:])
spots

,records,X,Y
0,19x5,5207.839,1161.146
1,20x5,5513.454,1160.421
2,18x6,4907.382,1448.196
3,19x6,5210.719,1448.793
4,20x6,5517.230,1458.521
...,...,...,...
292,9x26,2320.040,7249.660
293,8x26,2007.885,7254.641
294,7x26,1716.821,7256.204
295,8x27,2013.621,7540.370


In [ ]:
print(spots.X.min(), spots.X.max())
print(spots.Y.min(), spots.Y.max())

270.47 6692.643
1160.421 7546.201


In [ ]:
# histology image
from IPython.display import display, Image
display(Image(filename='bucket/HE_BC23209_C1.jpg',width=256, height = 256))

Output hidden; open in https://colab.research.google.com to view.

## Extracting features using ResNet50


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np


# Load the ResNet50 model pre-trained on ImageNet data
base_model = ResNet50(weights='imagenet', include_top=False, pooling = 'avg') #  pooling='avg')

# Choose the layer from which you want to get the features
# Let's try using the last layer before the fully connected layers
model = Model(inputs=base_model.input, outputs=base_model.output)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

img_path = 'bucket/images/HE_BC23209_C1.jpg'
preprocessed_img = preprocess_image(img_path)
print('Preprocessed image shape', preprocessed_img.shape)
def extract_features(preprocessed_img, model):
    features = model.predict(preprocessed_img)
    # Flatten the features to fit your embedding size requirement
    # For example, we flatten it and could use PCA or another method to reduce dimensionality if needed

    return features

# Extract features
features = extract_features(preprocessed_img, model)
print("Extracted features:", features.shape)

# from sklearn.decomposition import PCA

# # Initialize PCA to reduce the features to your desired size, e.g., 128
# pca = PCA(n_components=32, svd_solver = 'auto')

# # Fit PCA on the extracted features
# pca_result = pca.fit_transform(features.reshape(-1, 1))

# print("Reduced features shape:", pca_result.shape)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (91790429 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Preprocessed image shape (1, 224, 224, 3)
1/1 [==============================] - 1s 1s/step
Extracted features: (1, 2048)


## Creating windows for reviewing cells

In [ ]:
# image_path = 'test/data/hist2tscript/HE_BC23209_C1.tif'
# # read image
# image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
# # create windows for cells
# cells = []
# for index, cell in spots.iterrows():
#   x = round(cell['X'])
#   y = round(cell['Y'])

#   start_x = x - 112
#   start_y = y - 112

#   window = image[start_y:start_y + 224, start_x:start_x + 224]
#   preprocessed_window = preprocess_image(window)
#   cells.append(preprocessed_window)
# cells = np.array(cells)
# features = extract_features(cells, model)

# # Initialize PCA to reduce the features to your desired size, e.g., 128
# pca = PCA(n_components=32, svd_solver = 'randomized')

# # Fit PCA on the extracted features
# pca_result = pca.fit_transform(features)

# print("Reduced features shape:", pca_result.shape)

10/10 [==============================] - 16s 1s/step
Reduced features shape: (297, 32)


In [ ]:
import glob
import os
metadata = pd.read_csv('bucket/metadata.csv')

In [ ]:
metadata.value_counts(['type','patient']).to_frame().sort_index()

count
type             patient       
HER2_luminal     BC23287      3
                 BC23450      3
                 BC23901      2
                 BC23944      3
                 BC24220      3
HER2_non_luminal BC23567      3
                 BC23810      3
                 BC23903      3
                 BC24044      3
                 BC24105      3
Luminal_A        BC23268      3
                 BC23269      3
                 BC23272      3
                 BC24223      3
Luminal_B        BC23270      3
                 BC23277      3
                 BC23506      3
                 BC23508      3
                 BC23895      3
TNBC             BC23209      3
                 BC23288      3
                 BC23377      3
                 BC23803      3

In [ ]:
import pandas as pd
import numpy as np

# Assuming metadata is already loaded into a DataFrame named `metadata`
# and is structured as shown previously

# Unique types in the dataset
unique_types = metadata['type'].unique()

# Initialize empty lists to hold selected patients for training and testing
test_patients = []

# Select exactly one patient for each type for the testing set
for type_ in unique_types:
    # Filter metadata for the current type
    type_metadata = metadata[metadata['type'] == type_]

    # Select a random patient of this type
    patients = type_metadata['patient'].unique()
    selected_patient = np.random.choice(patients, 1)[0]

    # Add this patient to the test_patients list
    test_patients.append(selected_patient)

# Create the test_metadata DataFrame by filtering for selected test patients
test_metadata = metadata[metadata['patient'].isin(test_patients)]

# Create the train_metadata DataFrame by excluding the test patients
train_metadata = metadata[~metadata['patient'].isin(test_patients)]

# Output sizes to verify
print(f"Total entries: {len(metadata)}")
print(f"Training set entries: {len(train_metadata)}")
print(f"Testing set entries: {len(test_metadata)}")
train_metadata.to_csv('bucket/train_metadata.csv')
test_metadata.to_csv('bucket/test_metadata.csv')

Total entries: 68
Training set entries: 53
Testing set entries: 15


In [ ]:
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from sklearn.decomposition import PCA

# Load the ResNet50 model pre-trained on ImageNet data
base_model = ResNet50(weights='imagenet', include_top=False, pooling = 'avg') #  pooling='avg')

# Choose the layer from which you want to get the features
# Let's try using the last layer before the fully connected layers
model = Model(inputs=base_model.input, outputs=base_model.output)

def preprocess_image(img):
    return preprocess_input(img)

def extract_features(preprocessed_img, model):
    features = model.predict(preprocessed_img)
    # Flatten the features to fit your embedding size requirement
    # For example, we flatten it and could use PCA or another method to reduce dimensionality if needed

    return features

# Use train_data to train PCA for image embedding
data_path = 'bucket/spots/'
image_data_path = 'bucket/images/'
# train_spot_data = pd.concat([pd.read_csv(os.path.join(data_path, spot)) for spot in train_metadata['spot_coordinates']],ignore_index = True)
train_spots = []
for _ , record in train_metadata.iterrows():
  spot_data = pd.read_csv(os.path.join(data_path, record['spot_coordinates']))
  image = cv2.imread(os.path.join(image_data_path, record['histology_image'].replace('.jpg','.tif')), cv2.IMREAD_UNCHANGED)
  print(f"Reading image for Patient: {record['patient']}, Section: {record['replicate']}")
  for _ , spot in spot_data.iterrows():
    start_x = round(spot['X']) - 112
    start_y = round(spot['Y']) - 112

    window = image[start_y:start_y + 224, start_x:start_x + 224]
    preprocessed_window = preprocess_image(window)
    train_spots.append(window)
train_spots = np.array(train_spots)

Reading image for Patient: BC23287, Section: C1
Reading image for Patient: BC23287, Section: C2
Reading image for Patient: BC23287, Section: D1
Reading image for Patient: BC23450, Section: D2
Reading image for Patient: BC23450, Section: E1
Reading image for Patient: BC23450, Section: E2
Reading image for Patient: BC23901, Section: C2
Reading image for Patient: BC23901, Section: D1
Reading image for Patient: BC24220, Section: D2
Reading image for Patient: BC24220, Section: E1
Reading image for Patient: BC24220, Section: E2
Reading image for Patient: BC23567, Section: D2
Reading image for Patient: BC23567, Section: E1
Reading image for Patient: BC23567, Section: E2
Reading image for Patient: BC23810, Section: D2
Reading image for Patient: BC23810, Section: E1
Reading image for Patient: BC23810, Section: E2
Reading image for Patient: BC23903, Section: C1
Reading image for Patient: BC23903, Section: C2
Reading image for Patient: BC23903, Section: D1
Reading image for Patient: BC24044, Sect

In [ ]:
train_features = extract_features(train_spots, model)

759/759 [==============================] - 1186s 2s/step


In [ ]:
train_features.shape

(24274, 2048)

In [ ]:
# Initialize PCA to reduce the features to your desired size, e.g., 128
pca = PCA(n_components=32, svd_solver = 'randomized')

# Fit PCA on the extracted features
train_features_img_embed = pca.fit_transform(train_features)

print("Reduced features shape:", train_features_img_embed.shape)

Reduced features shape: (24274, 32)


In [ ]:
from joblib import dump, load
dump(pca, 'bucket/models/image_pca_resnet_32.joblib')
# load pca using the following command:
loaded_pca = load('bucket/models/image_pca_resnet_32.joblib')

In [ ]:
loaded_pca.components_.shape

(32, 2048)